## 1 Simulating a multi-link system

In [1]:
# symbolic computation tools
import sympy as sp
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi, lambdify
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify
from sympy import Symbol

from scipy.integrate import solve_ivp

from sympy.physics.mechanics import *

import numpy as np

from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')
sp.init_printing(use_latex=True);

import matplotlib.pyplot as plt
plt.ion()


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16

## 1.

In [2]:
(t, 
 theta_1, 
 theta_2, 
 theta_3, 
 l_1, 
 l_2, 
 l_3, 
 m_1,
 m_2,
 m_3,
 g) = symbols("""t, 
                         theta_1 
                         theta_2 
                         theta_3 
                         l_1 
                         l_2 
                         l_3
                         m_1
                         m_2
                         m_3
                         g""" , real = True)


theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

q1, q2, q3 = dynamicsymbols('q1 q2 q3')

In [3]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [4]:
T_01 = T(q1, 0, 0)
T_12 = T(q2, l_1, 0)
T_23 = T(q3, l_1+l_2, 0)

FK1 = (T_01 * Matrix([[l_1], [0], [1]]))[:-1,0]
FK2 = simplify((T_01 * T_12 * Matrix([[l_2], [0], [1]]))[:-1,0])
FK3 = simplify((T_01 * T_12 * T_23 * Matrix([[l_3], [0], [1]]))[:-1,0])

v1 = diff(FK1,Symbol('t'))
KE1 = (1/2)*m_1*(v1.T * v1)[0]

v2 = diff(FK2,Symbol('t'))
KE2 = (1/2)*m_2*(v2.T * v2)[0]
KE2

v3 = diff(FK3,Symbol('t'))
KE3 = (1/2)*m_3*(v3.T * v3)[0]
KE3

T = KE1 + KE2 + KE3
V = m_1*g*FK1[1] + m_2*g*FK2[1] + m_3*g*FK3[1]

L = T - V
L = simplify(L)
L


In [5]:
q = [q1, q2, q3]
l = LagrangesMethod(L, q)
le = l.form_lagranges_equations()
forcing = simplify(l.forcing)
mass = simplify(l.mass_matrix)
mass

⎡      2         ⎛  2                          2⎞      ⎛    2                 
⎢1.0⋅l₁ ⋅m₁ + m₂⋅⎝l₁  + 2⋅l₁⋅l₂⋅cos(q₂(t)) + l₂ ⎠ + m₃⋅⎝2⋅l₁ ⋅cos(q₂(t)) + 2⋅l
⎢                                                                             
⎢                                               ⎛  2                2         
⎢               l₂⋅m₂⋅(l₁⋅cos(q₂(t)) + l₂) + m₃⋅⎝l₁ ⋅cos(q₂(t)) + l₁  + l₁⋅l₂⋅
⎢                                                                             
⎢                                                                             
⎣                                                              1.0⋅l₃⋅m₃⋅(l₁⋅c

 2                                                                            
₁  + 2⋅l₁⋅l₂⋅cos(q₂(t)) + 2⋅l₁⋅l₂ + 2⋅l₁⋅l₃⋅cos(q₂(t) + q₃(t)) + 2⋅l₁⋅l₃⋅cos(q
                                                                              
                                                                         2    
cos(q₂(t)) + 2⋅l₁⋅l₂ + l₁⋅l₃⋅cos(q₂(t) + q₃(t)) + 2

## 2.

In [6]:
M =   lambdify((q1, q2, q3, Derivative(q1, Symbol('t')), Derivative(q2, Symbol('t')), Derivative(q3, Symbol('t')),
                l_1, l_2, l_3, m_1, m_2, m_3), mass)
EOM = lambdify((q1, q2, q3, Derivative(q1, Symbol('t')), Derivative(q2, Symbol('t')), Derivative(q3, Symbol('t')),
                l_1, l_2, l_3, m_1, m_2, m_3), forcing.subs(g,9.8))

In [7]:
M(0,0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1)

array([[21., 14.,  4.],
       [14., 10.,  3.],
       [ 4.,  3.,  1.]])

In [8]:
EOM(0, np.pi/2, 0, 10, 10, 10, 1, 1, 1, 1,1,1)

array([[1670.6],
       [-400. ],
       [-100. ]])

In [9]:
def dynamics(t, state, l_1, l_2, l_3, m_1, m_2, m_3):
    theta_1, theta_2, theta_3, dtheta_1, dtheta_2, dtheta_3 = state
    
    dydt = np.linalg.inv(M(theta_1, theta_2, theta_3, dtheta_1, dtheta_2, dtheta_3, l_1, l_2, l_3, m_1, m_2, m_3))@(EOM(theta_1, theta_2, theta_3, dtheta_1, dtheta_2, dtheta_3, l_1, l_2, l_3, m_1, m_2, m_3))

    return [dtheta_1, dtheta_2, dtheta_3, dydt[0][0], dydt[1][0], dydt[2][0]]

In [10]:
l1 = 1
l2 = 1
l3 = 1.5
m1 = 1
m2 = 1
m3 = 2


t_end = 10
dt = 0.0001
time = np.linspace(0,t_end,int(t_end/dt))
skip = 10

sol = solve_ivp(lambda t, y: dynamics(t, y, l1, l2, l3, m1, m2, m3),
                [0,t_end], [np.pi/2, 0, 0, 0, 0, 0], 
                t_eval = time, 
                rtol=1e-8, atol = 1e-8)

theta1 = sol.y[0,0:-1:skip]
theta2 = sol.y[1,0:-1:skip]
theta3 = sol.y[2,0:-1:skip]

## 3.

In [11]:
plt.clf()
plt.plot(sol.t, sol.y[0,:])
plt.plot(sol.t, sol.y[1,:])
plt.plot(sol.t, sol.y[2,:])
plt.ylabel('joint angles')
plt.xlabel('time (seconds)')
plt.legend(['theta1','theta2','theta3'])